In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urlencode, parse_qsl, urljoin
from typing import Tuple
from time import sleep

In [2]:
all_boardtype = {
    "wzine": "C",
    "album": "I",
    'list': "L",
}

# Set BaseUrl & BaseParameters & Base Headers

## base url & base_parameters

In [3]:
base_url = "https://cafe.naver.com/ArticleList.nhn"

In [4]:
club_id = "10050146" # joongonara_cafe's club id
num_of_list = 50
menu_id = 334 # notebook cgr
boardtype = all_boardtype['list']

In [5]:
base_params = {
    'search.clubid': club_id,
    'search.menuid': menu_id,
    'userDisplay': num_of_list,
    'search.boardtype': boardtype,
    'search.totalCount': '501',
    'search.cafeId': club_id,
    'search.page': 1
}

## base headers

In [6]:
urlparse("https://cafe.naver.com/joonggonara?articleid%3D962523238%2526referrerAllArticles%3Dfalse").query

'articleid%3D962523238%2526referrerAllArticles%3Dfalse'

In [7]:
base_headers = {
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'accept-encoding': 'gzip, deflate, br',
  'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
  'referer': 'https://cafe.naver.com/joonggonara?articleid%3D962523238%2526referrerAllArticles%3Dfalse',
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

# Scraping 요청하기

In [8]:
pending_list_q = []
data_list = []

seed_url = "https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&search.boardtype=L"

### session 만들기

In [9]:
s = requests.Session()
s

In [10]:
# initial
resp = s.get('https://cafe.naver.com/joonggonara', headers=base_headers)
resp

<Response [200]>

### 특정 메뉴에 요청하기 (initial)

In [11]:
seed_url = "https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.boardtype=L&search.menuid=334&search.marketBoardTab=D&search.specialmenutype=&userDisplay=50"
headers = base_headers.copy()
headers.update({'referer': resp.request.url})
init_resp = s.get(seed_url, headers=headers)
init_resp

<Response [200]>

### initial parsing

In [12]:
soup = BeautifulSoup(init_resp.text)

In [17]:
def list_page_parsing(soup):
  detail_tag_box = soup.select(".article-board.m-tcol-c")[-1]
  detail_tr_taglist = detail_tag_box.select('tbody > tr')
  data_list = []

  for detail_tr_tag in detail_tr_taglist:
    board_number = detail_tr_tag.select_one('.board-number').text
    board_content_tag = detail_tr_tag.select_one('.board-list a.article')
    content_url = board_content_tag.get('href')
    content_title = board_content_tag.text

    
    badge = board_content_tag.select_one('.list-i-selling')
    badge2 = board_content_tag.select_one('.list-i-selling-safe')
    
    if badge is not None:
      badge = badge.text
    if badge2 is not None:
      badge = badge2.text

    
    has_img = bool(detail_tr_tag.select_one('.list-i-img'))
    has_link = bool(detail_tr_tag.select_one('.list-i-link'))
    is_new = bool(detail_tr_tag.select_one('.list-i-new'))

    user_tag = detail_tr_tag.select_one('.p-nick')
    user_nickname = user_tag.text
    user_level = user_tag.select_one('.mem-level img').get('src')

    data_list.append({
        'board_num': board_number,
        'board_content_tag': str(board_content_tag),
        "content_url": content_url,
        "content_title": content_title.strip(),
        "badge": badge,
        "has_img": has_img,
        "has_link": has_link,
        "is_new": is_new,
        "user_nickname": user_nickname,
        "user_level": user_level
    })
  return data_list

In [18]:
len(data_list)

0

## next page 찾기

In [19]:
def find_next_page_url(soup):
  pagination_atag_list = soup.select('.prev-next a')
  for idx, pagination_atag in enumerate(pagination_atag_list):
    if "on" in pagination_atag.get('class', []):
      target_idx = idx+1
      break
  next_list_url = pagination_atag_list[target_idx].get('href')
  return next_list_url


# 실행할 코드

In [20]:
s = requests.Session()
# initial
resp = s.get('https://cafe.naver.com/joonggonara', headers=base_headers)

In [21]:
resp

<Response [200]>

In [22]:
parse_qsl(urlparse(next_url).query)

NameError: name 'next_url' is not defined

In [23]:
target_data_len = 10000

next_url = "https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.boardtype=L&search.menuid=334&search.marketBoardTab=D&search.specialmenutype=&userDisplay=50"
api_url = "https://apis.naver.com/cafe-web/cafe-articleapi/v2/cafes/10050146/articles/{board_num}?query=&menuId=334&boardType=L&useCafeId=true&requestFrom=A"

data_list = []
old_url = resp.request.url
while True:
  try:
    sleep(2.2)
    headers = base_headers.copy()
    headers.update({'referer': old_url})
    resp = s.get(next_url, headers=headers)
    list_soup = BeautifulSoup(resp.text)
    parsed_listing = list_page_parsing(list_soup)

    page_num = dict(parse_qsl(urlparse(next_url).query)).get('search.page', 1)

    old_url = next_url
    print("현재 페이지:", page_num, "   ", old_url)

    print('detail_page:', end=" ")
    for parsed_item in parsed_listing:
      detail_visible_url = urljoin(resp.request.url, parsed_item['content_url'])
      headers.update({'referer': detail_visible_url, 'accept': 'application/json, text/plain, */*', 'x-cafe-product': 'pc'})
      detail_api_url = api_url.format(board_num=parsed_item['board_num'])
      detail_api_resp = s.get(detail_api_url, headers=headers)

      
      parsed_item['detail_status'] = detail_api_resp.status_code
      parsed_item['detail_json'] = detail_api_resp.json()
      print(parsed_item['board_num'], end=", ")
      
      sleep(0.3)
      data_list.append(parsed_item)
    
    next_url = find_next_page_url(list_soup)
    next_url = urljoin(resp.request.url, next_url)
    print()
    if old_url == next_url:
      print("에러 발생", old_url, next_url, sep="\n")
      break
    
    if len(data_list) >= target_data_len:
      print("목표 달성 멈춤", len(data_list))
      break
    print("현재 페이지:", page_num, "   ", old_url)
  except:
    sleep(3)



현재 페이지: 1     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.boardtype=L&search.menuid=334&search.marketBoardTab=D&search.specialmenutype=&userDisplay=50
detail_page: 966933427, 966933302, 966933267, 966933262, 966933203, 966933199, 966933198, 966933194, 966933017, 966933004, 966932873, 966932852, 966932593, 966932539, 966932525, 966932489, 966932455, 966932334, 966932258, 966932204, 966932103, 966931999, 966931905, 966931775, 966931758, 966931739, 966931720, 966931640, 966931546, 966931531, 966931525, 966931507, 966931484, 966931458, 966931384, 966931324, 966931182, 966931119, 966931093, 966931045, 966930964, 966930953, 966930947, 966930874, 966930856, 966930804, 966930784, 966930759, 966930754, 966930696, 
현재 페이지: 1     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.boardtype=L&search.menuid=334&search.marketBoardTab=D&search.specialmenutype=&userDisplay=50
현재 페이지: 2     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.m

detail_page: 966883731, 966883490, 966883437, 966883263, 966882866, 966882670, 966882631, 966882516, 966882431, 966882136, 966881970, 966881579, 966881516, 966881143, 966881079, 966880935, 966880603, 966880522, 966880218, 966880026, 966879769, 966879647, 966879484, 966878987, 966877768, 966877324, 966876958, 966876514, 966876469, 966875956, 966875594, 966875517, 966875492, 966875292, 966875276, 966875266, 966875240, 966875054, 966874405, 966874400, 966874357, 966874072, 966874068, 966873991, 966873910, 966873863, 966873747, 966873653, 966873634, 966873388, 
현재 페이지: 17     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=1001&search.cafeId=10050146&search.page=17
현재 페이지: 18     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=1001&search.cafeId=10050146&search.page=18
detail_p

detail_page: 966802623, 966802491, 966802364, 966802043, 966801239, 966801225, 966801210, 966801194, 966801175, 966800978, 966800969, 966800656, 966800487, 966800457, 966800308, 966800116, 966800111, 966800020, 966799880, 966799848, 966799757, 966799446, 966799324, 966799264, 966799121, 966798996, 966798840, 966798770, 966798674, 966798567, 966798398, 966798285, 966798153, 966798148, 966798110, 966798104, 966797989, 966797972, 966797896, 966797752, 966797748, 966797699, 966797683, 966797498, 966797493, 966797409, 966797309, 966797122, 966797084, 966796834, 
현재 페이지: 33     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=2001&search.cafeId=10050146&search.page=33
현재 페이지: 34     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=2001&search.cafeId=10050146&search.page=34
detail_p

detail_page: 966680307, 966680291, 966679878, 966679863, 966679706, 966679553, 966678839, 966678620, 966678185, 966678040, 966677953, 966677200, 966677044, 966676739, 966676703, 966676607, 966676205, 966675902, 966675719, 966675472, 966675147, 966675083, 966674666, 966674621, 966674341, 966674241, 966673807, 966673557, 966673541, 966673367, 966673323, 966673234, 966673125, 966672508, 966672487, 966672483, 966672471, 966672043, 966671905, 966671836, 966671822, 966671665, 966671651, 966671538, 966671534, 966671367, 966671313, 966671262, 966671163, 966670932, 
현재 페이지: 49     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=2501&search.cafeId=10050146&search.page=49
현재 페이지: 50     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=2501&search.cafeId=10050146&search.page=50
detail_p

detail_page: 966569804, 966569726, 966569617, 966569138, 966569047, 966568938, 966568925, 966568918, 966568817, 966568739, 966568738, 966568223, 966568147, 966568043, 966567496, 966567458, 966567456, 966567188, 966567033, 966566294, 966565948, 966565725, 966565643, 966565417, 966565369, 966565287, 966565152, 966565007, 966564657, 966564574, 966564501, 966564388, 966564345, 966563883, 966563868, 966563486, 966563368, 966563089, 966562703, 966562628, 966562534, 966562460, 966562399, 966562357, 966562177, 966562083, 966561946, 966561697, 966561597, 966561525, 
현재 페이지: 65     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=3501&search.cafeId=10050146&search.page=65
현재 페이지: 66     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=3501&search.cafeId=10050146&search.page=66
detail_p

...

detail_page: 965983833, 965983405, 965983113, 965983040, 965982921, 965982718, 965982630, 965982060, 965981985, 965981845, 965981432, 965981143, 965980987, 965980932, 965980800, 965980746, 965980249, 965980236, 965980209, 965980182, 965980144, 965979822, 965979680, 965979586, 965979304, 965979253, 965979170, 965979080, 965979010, 965978849, 965978326, 965977950, 965977689, 965977190, 965976985, 965976792, 965976178, 965975847, 965975795, 965975613, 965975604, 965975098, 965975073, 965974897, 965974764, 965974554, 965974367, 965974362, 965974317, 965973914, 
현재 페이지: 129     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=6501&search.cafeId=10050146&search.page=129
현재 페이지: 130     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=6501&search.cafeId=10050146&search.page=130
deta

detail_page: 965810896, 965810481, 965810415, 965810075, 965809846, 965809825, 965809634, 965809617, 965809470, 965809238, 965808090, 965807877, 965807873, 965807553, 965807152, 965806938, 965806803, 965806743, 965806728, 965806723, 965806568, 965806378, 965805914, 965805304, 965805260, 965805212, 965804559, 965804471, 965803811, 965803744, 965803711, 965803684, 965803655, 965803383, 965803178, 965802804, 965802383, 965802266, 965802061, 965801605, 965801441, 965801304, 965800813, 965800721, 965800525, 965800052, 965799933, 965799667, 965799563, 965799561, 
현재 페이지: 145     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=7501&search.cafeId=10050146&search.page=145
현재 페이지: 146     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=7501&search.cafeId=10050146&search.page=146
deta

detail_page: 965607988, 965607900, 965607819, 965607658, 965607526, 965607453, 965607315, 965607069, 965606773, 965606584, 965606542, 965606303, 965606158, 965606078, 965606040, 965606039, 965605697, 965605628, 965605541, 965605091, 965604619, 965604572, 965604510, 965604446, 965604181, 965603366, 965603355, 965603263, 965603099, 965603067, 965603062, 965602977, 965602933, 965602728, 965602594, 965602555, 965602098, 965601755, 965601502, 965601253, 965601235, 965600954, 965600440, 965600313, 965600177, 965600124, 965599667, 965599477, 965599240, 965599237, 
현재 페이지: 161     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=8001&search.cafeId=10050146&search.page=161
현재 페이지: 162     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=8501&search.cafeId=10050146&search.page=162
deta

detail_page: 965403579, 965403450, 965402962, 965402950, 965402867, 965402846, 965402836, 965402630, 965402517, 965402286, 965402199, 965402098, 965402037, 965401842, 965401630, 965401143, 965401084, 965401072, 965400911, 965400898, 965400885, 965400863, 965400857, 965400703, 965400685, 965400375, 965399795, 965399553, 965399508, 965399499, 965399435, 965399050, 965398914, 965398820, 965398766, 965398376, 965398279, 965398243, 965398105, 965397801, 965397685, 965397565, 965397070, 965396984, 965396921, 965396752, 965396665, 965396640, 965396612, 965396185, 
현재 페이지: 177     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=9001&search.cafeId=10050146&search.page=177
현재 페이지: 178     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=9001&search.cafeId=10050146&search.page=178
deta

detail_page: 965204978, 965204628, 965204435, 965204263, 965204181, 965204130, 965204090, 965203962, 965203856, 965203798, 965203760, 965203747, 965203488, 965203453, 965203360, 965203079, 965203062, 965202984, 965202953, 965202818, 965202792, 965202746, 965202684, 965202675, 965202364, 965202263, 965201780, 965201662, 965201545, 965201370, 965201328, 965201224, 965201050, 965200940, 965200832, 965200557, 965200465, 965200424, 965200408, 965200378, 965200364, 965200300, 965200270, 965200256, 965199979, 965199911, 965199888, 965199689, 965199515, 965199257, 
현재 페이지: 193     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=10001&search.cafeId=10050146&search.page=193
현재 페이지: 194     https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=334&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=10001&search.cafeId=10050146&search.page=194
de

In [27]:
import pandas as pd

In [28]:
sample = pd.DataFrame()
sample

""


In [29]:
dir(sample)

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__dataframe__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex_

In [30]:
len(data_list)

10000

In [36]:
import pandas as pd
df = pd.DataFrame(data_list)
df.to_csv("naver_joongonara_notebook.csv")

In [32]:
import json

In [33]:
with open("naver_joongonara_notebook.json", 'wt') as f:
  f.write(json.dumps(data_list))

In [34]:
str(data_list[0]['board_content_tag'])

'<a class="article" href="/ArticleRead.nhn?clubid=10050146&amp;page=1&amp;userDisplay=50&amp;menuid=334&amp;boardtype=L&amp;articleid=966933427&amp;referrerAllArticles=false" onclick="clickcr(this, \'gdb.title\',\'\',\'\',event);">\n                                \n                                \n\n                                \n                                \n\n                                \n                                \n                                    \n                                    LG전자 2021 그램16 16ZD90P-GX5TK (i5/16GB)\n                                    \n                                \n                            </a>'

In [35]:
for data in data_list:
  data['board_content_tag'] = str(data['board_content_tag'])

In [38]:
df.columns

Index(['board_num', 'board_content_tag', 'content_url', 'content_title',
       'badge', 'has_img', 'has_link', 'is_new', 'user_nickname', 'user_level',
       'detail_status', 'detail_json'],
      dtype='object')

In [41]:
df['detail_json']

0       {'result': {'cafeId': 10050146, 'articleId': 9...
1       {'result': {'cafeId': 10050146, 'articleId': 9...
2       {'result': {'errorCode': '0004', 'reason': '로그...
3       {'result': {'cafeId': 10050146, 'articleId': 9...
4       {'result': {'cafeId': 10050146, 'articleId': 9...
                              ...                        
9995    {'result': {'cafeId': 10050146, 'articleId': 9...
9996    {'result': {'cafeId': 10050146, 'articleId': 9...
9997    {'result': {'cafeId': 10050146, 'articleId': 9...
9998    {'result': {'cafeId': 10050146, 'articleId': 9...
9999    {'result': {'errorCode': '0004', 'reason': '로그...
Name: detail_json, Length: 10000, dtype: object

In [45]:
df

,board_num,board_content_tag,content_url,content_title,badge,has_img,has_link,is_new,user_nickname,user_level,detail_status,detail_json
0,966933427,"<a class=""article"" href=""/ArticleRead.nhn?club...",/ArticleRead.nhn?clubid=10050146&page=1&userDi...,LG전자 2021 그램16 16ZD90P-GX5TK (i5/16GB),None,True,False,True,베이직스,https://cafe.pstatic.net/levelicon/1/1_150.gif,200,"{'result': {'cafeId': 10050146, 'articleId': 9..."
1,966933302,"<a class=""article"" href=""/ArticleRead.nhn?club...",/ArticleRead.nhn?clubid=10050146&page=1&userDi...,MK193KH/A 새제품 팝니다.,None,True,False,True,새까만별,https://cafe.pstatic.net/levelicon/1/1_1.gif,200,"{'result': {'cafeId': 10050146, 'articleId': 9..."
2,966933267,"<a class=""article"" href=""/ArticleRead.nhn?club...",/ArticleRead.nhn?clubid=10050146&page=1&userDi...,[미개봉]LG 그램14 14Z95N-GP5NL i5/16GB 노트북 판매,None,True,True,True,type0214,https://cafe.pstatic.net/levelicon/1/1_150.gif,401,"{'result': {'errorCode': '0004', 'reason': '로그..."
3,966933262,"<a class=""article"" href=""/ArticleRead.nhn?club...",/ArticleRead.nhn?clubid=10050146&page=1&userDi...,2021 맥북프로 레티나 엠원 m1pro m1max 프로 맥스 14인치 판매,None,True,False,True,topontopcom,https://cafe.pstatic.net/levelicon/1/1_150.gif,200,"{'result': {'cafeId': 10050146, 'articleId': 9..."
4,966933203,"<a class=""article"" href=""/ArticleRead.nhn?club...",/ArticleRead.nhn?clubid=10050146&page=1&userDi...,엘지 14u530 i5노트북팝니다.,None,True,False,True,블루모닝,https://cafe.pstatic.net/levelicon/1/1_1.gif,200,"{'result': {'cafeId': 10050146, 'articleId': 9..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,965081979,"<a class=""article"" href=""/ArticleRead.nhn?club...",/ArticleRead.nhn?clubid=10050146&page=200&user...,부산 2022 맥북 에어 m1 16g 513gb 스그 풀박스 팝니다,None,True,False,False,xelloss4088,https://cafe.pstatic.net/levelicon/1/1_1.gif,200,"{'result': {'cafeId': 10050146, 'articleId': 9..."
9996,965081960,"<a class=""article"" href=""/ArticleRead.nhn?club...",/ArticleRead.nhn?clubid=10050146&page=200&user...,노트북 LENOVO IDEAPAD 330-15IKB 15.6인치/i5-8250U/1...,None,True,False,False,soju3809,https://cafe.pstatic.net/levelicon/1/1_1.gif,200,"{'result': {'cafeId': 10050146, 'articleId': 9..."
9997,965081869,"<a class=""article"" href=""/ArticleRead.nhn?club...",/ArticleRead.nhn?clubid=10050146&page=200&user...,노트북 LENOVO IDEAPAD 330S-14IKB 14.1인치/i5-8250U/...,None,True,False,False,soju3809,https://cafe.pstatic.net/levelicon/1/1_1.gif,200,"{'result': {'cafeId': 10050146, 'articleId': 9..."
9998,965081469,"<a class=""article"" href=""/ArticleRead.nhn?club...",/ArticleRead.nhn?clubid=10050146&page=200&user...,맥북프로 16인치 1테라 16램,None,True,False,False,삼재자르,https://cafe.pstatic.net/levelicon/1/1_1.gif,200,"{'result': {'cafeId': 10050146, 'articleId': 9..."
